In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
MAIN_PATH = '/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/'

## 데이터 준비하기
- prepare_custom_dataset
### directories and files for yours datsaets
/image_folder

├── 0_0_0000000

│   ├── 0_0.jpg

│   ├── 0_1.jpg

│   ├── 0_2.jpg

│   ├── 0_3.jpg

│   └── 0_4.jpg

├── 0_0_0000001

│   ├── 0_5.jpg

│   ├── 0_6.jpg

│   ├── 0_7.jpg

│   ├── 0_8.jpg

│   └── 0_9.jpg

In [5]:
!pip install mxnet
!apt-get update
!apt-get install -y ffmpeg libsm6 libxext6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libs

In [7]:
!python -m mxnet.tools.im2rec --list --recursive train /content/drive/MyDrive/dataset
!python -m mxnet.tools.im2rec --num-thread 16 --quality 100 train /content/drive/MyDrive/dataset

Creating .rec file from /content/train.lst in /content


In [8]:
!pwd

/content


In [2]:
import shutil

source_path = '/content/train.idx'
destination_path = MAIN_PATH + 'arcface_torch/train_tmp/ms1m-retinaface-t1/train.idx'

shutil.move(source_path, destination_path)

FileNotFoundError: ignored

In [11]:
import shutil

source_path = '/content/train.lst'
destination_path = MAIN_PATH + 'arcface_torch/train_tmp/ms1m-retinaface-t1/train.lst'

shutil.move(source_path, destination_path)

'/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/Pytorch_ArcFace/arcface_torch/train_tmp/ms1m-retinaface-t1/train.lst'

In [12]:
import shutil

source_path = '/content/train.rec'
destination_path = MAIN_PATH + 'arcface_torch/train_tmp/ms1m-retinaface-t1/train.rec'

shutil.move(source_path, destination_path)

'/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/Pytorch_ArcFace/arcface_torch/train_tmp/ms1m-retinaface-t1/train.rec'

# reference
## ArcFace
- https://github.com/deepinsight/insightface/tree/master/recognition/arcface_torch

In [7]:
!pip install scipy==1.8.1
#설치하고 런타임 재시작해야됨

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


### import modules

In [8]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [9]:
#arcface_torch 파일이 있는 경로
import sys
sys.path.append(MAIN_PATH +'Pytorch_ArcFace/arcface_torch')

In [10]:
import argparse
import logging
import os
from datetime import datetime

import numpy as np
import torch
from backbones import get_model
from dataset import get_dataloader
from losses import CombinedMarginLoss, ArcFace, CosFace
from lr_scheduler import PolynomialLRWarmup
from partial_fc_v2 import PartialFC_V2
from torch import distributed
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.utils_callbacks import CallBackLogging, CallBackVerification
from utils.utils_config import get_config
from utils.utils_distributed_sampler import setup_seed
from utils.utils_logging import AverageMeter, init_logging
from torch.distributed.algorithms.ddp_comm_hooks.default_hooks import fp16_compress_hook

assert torch.__version__ >= "1.12.0", "In order to enjoy the features of the new torch, \
we have upgraded the torch to 1.12.0. torch before than 1.12.0 may not work in the future."

try:
    rank = int(os.environ["RANK"])
    local_rank = int(os.environ["LOCAL_RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    distributed.init_process_group("nccl")
except KeyError:
    rank = 0
    local_rank = 0
    world_size = 1
    distributed.init_process_group(
        backend="nccl",
        init_method="tcp://127.0.0.1:12584",
        rank=rank,
        world_size=world_size,
    )

/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:125: UserWarning: Decorating classes is deprecated and will be disabled in future versions. You should only decorate functions or methods. To preserve the current behavior of class decoration, you can directly decorate the `__init__` method and nothing else.
  warnings.warn("Decorating classes is deprecated and will be disabled in "


In [11]:
torch.backends.cudnn.benchmark = True

### config 파일 -> 'configs/ms1mv3_r50_onegpu'

- config.margin_list = (1.0, 0.5, 0.0)
- config.network = "r50" #backbone network
- config.resume = False
- config.output = None
- config.embedding_size = 512
- config.sample_rate = 1.0
- config.fp16 = True
- config.momentum = 0.9
- config.weight_decay = 5e-4
- config.batch_size = 128
- config.lr = 0.02
- config.verbose = 2000
- config.dali = False

- config.rec = "/train_tmp/ms1m-retinaface-t1" #dataset이 있는 경로
- config.num_classes = 93431 #수정 필요
- config.num_image = 5179510 # 수정 필요
- config.num_epoch = 20
- config.warmup_epoch = 0
- config.val_targets = ['lfw', 'cfp_fp', "agedb_30"]


In [12]:
cfg = get_config('configs/mydata_onegpu')

In [20]:
cfg.network = "r18"
cfg.optimizer = "adamw"
cfg.batch_size = 32
cfg.batch_size
cfg.num_epoch

32

In [21]:
setup_seed(seed=cfg.seed, cuda_deterministic=False)

In [22]:
torch.cuda.set_device(local_rank)

os.makedirs(cfg.output, exist_ok=True)
init_logging(rank, cfg.output)

INFO:root:rank_id: 0


Training: 2023-06-19 03:36:58,402-rank_id: 0
Training: 2023-06-19 03:36:58,402-rank_id: 0


In [23]:
cfg.network

'r18'

In [24]:
cfg.num_epoch

20

In [25]:
cfg.save_all_states

True

In [26]:
cfg.resume

True

In [27]:
summary_writer = (
        SummaryWriter(log_dir=os.path.join(cfg.output, "tensorboard"))
        if rank == 0
        else None
    )

### using_wandb

In [28]:
cfg.using_wandb

False

In [29]:
cfg.network

'r18'

In [30]:
cfg.num_classes

165

In [31]:
cfg.num_image

37599

In [32]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ea26e98fdb970f55aece145a66b1cd4783ec5924596014c0ff2987e3a019e330
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [33]:
wandb_logger = None
if cfg.using_wandb:
    import wandb
    # Sign in to wandb
    try:
        wandb.login(key=cfg.wandb_key)
    except Exception as e:
        print("WandB Key must be provided in config file (base.py).")
        print(f"Config Error: {e}")
    # Initialize wandb
    run_name = datetime.now().strftime("%y%m%d_%H%M") + f"_GPU{rank}"
    run_name = run_name if cfg.suffix_run_name is None else run_name + f"_{cfg.suffix_run_name}"
    try:
        wandb_logger = wandb.init(
            entity = cfg.wandb_entity,
            project = cfg.wandb_project,
            sync_tensorboard = True,
            resume=cfg.wandb_resume,
            name = run_name,
            notes = cfg.notes) if rank == 0 or cfg.wandb_log_all else None
        if wandb_logger:
            wandb_logger.config.update(cfg)
    except Exception as e:
        print("WandB Data (Entity and Project name) must be provided in config file (base.py).")
        print(f"Config Error: {e}")

### trainloader(수정 필요)

In [34]:
#batch_size 크면 메모리 오류 뜸
cfg.batch_size

32

In [35]:
train_loader = get_dataloader(
    cfg.rec,
    local_rank,
    cfg.batch_size,
    cfg.dali,
    cfg.dali_aug,
    cfg.seed,
    cfg.num_workers
)

In [36]:
cfg.rec #dataset이 있는 경로

'/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/Pytorch_ArcFace/arcface_torch/train_tmp/ms1m-retinaface-t1'

### backbone

In [37]:
backbone = get_model(
        cfg.network, dropout=0.0, fp16=cfg.fp16, num_features=cfg.embedding_size).cuda()

# 사전 훈련 모델 불러오기
weight_path = MAIN_PATH + 'Pytorch_ArcFace/arcface_torch/weights/backbone_r18_arc.pth'
dict_checkpoint = torch.load(weight_path)
backbone.load_state_dict(dict_checkpoint)

backbone = torch.nn.parallel.DistributedDataParallel(
    module=backbone, broadcast_buffers=False, device_ids=[local_rank], bucket_cap_mb=16,
    find_unused_parameters=True)
backbone.register_comm_hook(None, fp16_compress_hook)

backbone.train()
# FIXME using gradient checkpoint if there are some unused parameters will cause error
backbone._set_static_graph()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/distributed.py:1915: UserWarning: You passed find_unused_parameters=true to DistributedDataParallel, `_set_static_graph` will detect unused parameters automatically, so you do not need to set find_unused_parameters=true, just be sure these unused parameters will not change during training loop while calling `_set_static_graph`.
  warnings.warn(


### margin loss

In [38]:
# # CombinedMarginLoss
# margin_loss = CombinedMarginLoss(
#         64,
#         cfg.margin_list[0],
#         cfg.margin_list[1],
#         cfg.margin_list[2],
#         cfg.interclass_filtering_threshold
#     )

# # ArcFace
# margin_loss = ArcFace()

# CosFace
margin_loss = CosFace()

### module_partial_fc

In [39]:
if cfg.optimizer == "sgd":
    module_partial_fc = PartialFC_V2(
        margin_loss, cfg.embedding_size, cfg.num_classes,
        cfg.sample_rate, False)
    module_partial_fc.train().cuda()
    # TODO the params of partial fc must be last in the params list
    opt = torch.optim.SGD(
        params=[{"params": backbone.parameters()}, {"params": module_partial_fc.parameters()}],
        lr=cfg.lr, momentum=0.9, weight_decay=cfg.weight_decay)

elif cfg.optimizer == "adamw":
    module_partial_fc = PartialFC_V2(
        margin_loss, cfg.embedding_size, cfg.num_classes,
        cfg.sample_rate, False)
    module_partial_fc.train().cuda()
    opt = torch.optim.AdamW(
        params=[{"params": backbone.parameters()}, {"params": module_partial_fc.parameters()}],
        lr=cfg.lr, weight_decay=cfg.weight_decay)
else:
    raise

In [40]:
cfg.optimizer

'adamw'

### hyper parameters

In [41]:
cfg.total_batch_size = cfg.batch_size * world_size
cfg.warmup_step = cfg.num_image // cfg.total_batch_size * cfg.warmup_epoch
cfg.total_step = cfg.num_image // cfg.total_batch_size * cfg.num_epoch

lr_scheduler = PolynomialLRWarmup(
    optimizer=opt,
    warmup_iters=cfg.warmup_step,
    total_iters=cfg.total_step)

start_epoch = 0
global_step = 0

In [42]:
cfg.total_batch_size

32

In [43]:
cfg.warmup_step

0

In [44]:
cfg.output

'work_dirs/mydata_onegpu'

In [45]:
cfg.resume

True

In [46]:
cfg.batch_size

32

### checkpoint

#### 학습 다시 재개하고 싶을 때

In [33]:
model_path = '/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/Pytorch_ArcFace/arcface_torch/r34_Combined_sgd/mydata_onegpu/model.pt'

In [ ]:
dict_checkpoint = torch.load(model_path)

In [ ]:
dict_checkpoint.keys()

In [ ]:
if cfg.resume:
    dict_checkpoint = torch.load(os.path.join(cfg.output, f"checkpoint_gpu_{rank}.pt"))
    start_epoch = dict_checkpoint["epoch"]
    global_step = dict_checkpoint["global_step"]
    backbone.module.load_state_dict(dict_checkpoint["state_dict_backbone"])
    module_partial_fc.load_state_dict(dict_checkpoint["state_dict_softmax_fc"])
    opt.load_state_dict(dict_checkpoint["state_optimizer"])
    lr_scheduler.load_state_dict(dict_checkpoint["state_lr_scheduler"])
    del dict_checkpoint


In [ ]:
cfg.keys()

### Valid 로드

In [47]:
for key, value in cfg.items():
    num_space = 25 - len(key)
    logging.info(": " + key + " " * num_space + str(value))

INFO:root:: margin_list              [1.0, 0.5, 0.0]


Training: 2023-06-19 03:37:54,915-: margin_list              [1.0, 0.5, 0.0]
Training: 2023-06-19 03:37:54,915-: margin_list              [1.0, 0.5, 0.0]


INFO:root:: network                  r18


Training: 2023-06-19 03:37:54,920-: network                  r18
Training: 2023-06-19 03:37:54,920-: network                  r18


INFO:root:: resume                   True


...

Training: 2023-06-19 03:37:55,083-: total_batch_size         32
Training: 2023-06-19 03:37:55,083-: total_batch_size         32


INFO:root:: warmup_step              0


Training: 2023-06-19 03:37:55,086-: warmup_step              0
Training: 2023-06-19 03:37:55,086-: warmup_step              0


INFO:root:: total_step               23480


Training: 2023-06-19 03:37:55,090-: total_step               23480
Training: 2023-06-19 03:37:55,090-: total_step               23480


In [48]:
callback_verification = CallBackVerification(
    val_targets=cfg.val_targets, rec_prefix=cfg.rec,
    summary_writer=summary_writer, wandb_logger = wandb_logger
)
callback_logging = CallBackLogging(
    frequent=cfg.frequent,
    total_step=cfg.total_step,
    batch_size=cfg.batch_size,
    start_step = global_step,
    writer=summary_writer
)

loss_am = AverageMeter()
amp = torch.cuda.amp.grad_scaler.GradScaler(growth_interval=100)

loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
torch.Size([12000, 3, 112, 112])
loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
torch.Size([14000, 3, 112, 112])
loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
torch.Size([12000, 3, 112, 112])


In [49]:
callback_verification.ver_name_list

['lfw', 'cfp_fp', 'agedb_30']

### train

In [50]:
cfg.num_epoch

20

In [51]:
for _, (img, local_labels) in enumerate(train_loader):
  break
img.size()

torch.Size([32, 3, 112, 112])

In [52]:
cfg.output

'work_dirs/mydata_onegpu'

In [53]:
import scipy
print(scipy.__version__)

1.8.1


In [54]:
#degrade scipy (callback_verification(global_step, backbone) 여기서 ValueError: Expect x to not have duplicates 발생)
!pip install scipy==1.8.1
#설치하고 런타임 재시작해야됨

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
#degrade 됐는지 확인
import scipy
print(scipy.__version__)

1.8.1


In [56]:
for epoch in range(start_epoch, cfg.num_epoch):

    if isinstance(train_loader, DataLoader):
        train_loader.sampler.set_epoch(epoch)
    for _, (img, local_labels) in enumerate(train_loader):
        global_step += 1
        local_embeddings = backbone(img)
        loss: torch.Tensor = module_partial_fc(local_embeddings, local_labels)

        if cfg.fp16:
            amp.scale(loss).backward()
            if global_step % cfg.gradient_acc == 0:
                amp.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(backbone.parameters(), 5)
                amp.step(opt)
                amp.update()
                opt.zero_grad()
        else:
            loss.backward()
            if global_step % cfg.gradient_acc == 0:
                torch.nn.utils.clip_grad_norm_(backbone.parameters(), 5)
                opt.step()
                opt.zero_grad()
        lr_scheduler.step()

        with torch.no_grad():
            if wandb_logger:
                wandb_logger.log({
                    'Loss/Step Loss': loss.item(),
                    'Loss/Train Loss': loss_am.avg,
                    'Process/Step': global_step,
                    'Process/Epoch': epoch
                })

            loss_am.update(loss.item(), 1)
            callback_logging(global_step, loss_am, epoch, cfg.fp16, lr_scheduler.get_last_lr()[0], amp)

            if global_step % cfg.verbose == 0 and global_step > 0:
                callback_verification(global_step, backbone)

    if cfg.save_all_states:
        checkpoint = {
            "epoch": epoch + 1,
            "global_step": global_step,
            "state_dict_backbone": backbone.module.state_dict(),
            "state_dict_softmax_fc": module_partial_fc.state_dict(),
            "state_optimizer": opt.state_dict(),
            "state_lr_scheduler": lr_scheduler.state_dict()
        }
        torch.save(checkpoint, os.path.join(cfg.output, f"checkpoint_gpu_{rank}.pt"))

    if rank == 0:
        path_module = os.path.join(cfg.output, "model.pt")
        torch.save(backbone.module.state_dict(), path_module)

        if wandb_logger and cfg.save_artifacts:
            artifact_name = f"{run_name}_E{epoch}"
            model = wandb.Artifact(artifact_name, type='model')
            model.add_file(path_module)
            wandb_logger.log_artifact(model)

    if cfg.dali:
        train_loader.reset()

if rank == 0:
    path_module = os.path.join(cfg.output, "model.pt")
    torch.save(backbone.module.state_dict(), path_module)

    if wandb_logger and cfg.save_artifacts:
        artifact_name = f"{run_name}_Final"
        model = wandb.Artifact(artifact_name, type='model')
        model.add_file(path_module)
        wandb_logger.log_artifact(model)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
INFO:torch.nn.parallel.distributed:Reducer buckets have been rebuilt in this iteration.


Training: 2023-06-19 03:39:19,509-Reducer buckets have been rebuilt in this iteration.
Training: 2023-06-19 03:39:19,509-Reducer buckets have been rebuilt in this iteration.


INFO:root:Speed 395.06 samples/sec   Loss 43.5205   LearningRate 0.019983   Epoch: 0   Global Step: 20   Fp16 Grad Scale: 1024   Required: 6 hours


Training: 2023-06-19 03:39:21,018-Speed 395.06 samples/sec   Loss 43.5205   LearningRate 0.019983   Epoch: 0   Global Step: 20   Fp16 Grad Scale: 1024   Required: 6 hours
Training: 2023-06-19 03:39:21,018-Speed 395.06 samples/sec   Loss 43.5205   LearningRate 0.019983   Epoch: 0   Global Step: 20   Fp16 Grad Scale: 1024   Required: 6 hours


INFO:root:Speed 363.52 samples/sec   Loss 41.7087   LearningRate 0.019974   Epoch: 0   Global Step: 30   Fp16 Grad Scale: 1024   Required: 4 hours


...

Training: 2023-06-19 04:26:21,243-Speed 383.70 samples/sec   Loss 0.0278   LearningRate 0.000017   Epoch: 19   Global Step: 23460   Fp16 Grad Scale: 65536   Required: 0 hours
Training: 2023-06-19 04:26:21,243-Speed 383.70 samples/sec   Loss 0.0278   LearningRate 0.000017   Epoch: 19   Global Step: 23460   Fp16 Grad Scale: 65536   Required: 0 hours


INFO:root:Speed 378.38 samples/sec   Loss 0.0067   LearningRate 0.000009   Epoch: 19   Global Step: 23470   Fp16 Grad Scale: 65536   Required: 0 hours


Training: 2023-06-19 04:26:22,095-Speed 378.38 samples/sec   Loss 0.0067   LearningRate 0.000009   Epoch: 19   Global Step: 23470   Fp16 Grad Scale: 65536   Required: 0 hours
Training: 2023-06-19 04:26:22,095-Speed 378.38 samples/sec   Loss 0.0067   LearningRate 0.000009   Epoch: 19   Global Step: 23470   Fp16 Grad Scale: 65536   Required: 0 hours


INFO:root:Speed 383.67 samples/sec   Loss 0.0148   LearningRate 0.000000   Epoch: 19   Global Step: 23480   Fp16 Grad Scale: 65536   Required: -0 hours


Training: 2023-06-19 04:26:22,935-Speed 383.67 samples/sec   Loss 0.0148   LearningRate 0.000000   Epoch: 19   Global Step: 23480   Fp16 Grad Scale: 65536   Required: -0 hours
Training: 2023-06-19 04:26:22,935-Speed 383.67 samples/sec   Loss 0.0148   LearningRate 0.000000   Epoch: 19   Global Step: 23480   Fp16 Grad Scale: 65536   Required: -0 hours


In [57]:
import shutil

source_path = '/content/work_dirs'

# destination_path = MAIN_PATH + 'Pytorch_ArcFace/arcface_torch/r18_Combined_sgd' # r18_Combined_sgd
# destination_path = MAIN_PATH + 'Pytorch_ArcFace/arcface_torch/r34_Combined_sgd_20ep_resize' # r34_Combined_sgd_20ep_resize
# destination_path = MAIN_PATH + 'Pytorch_ArcFace/arcface_torch/r34_Combined_sgd_20ep_resize_batch32' # r34_Combined_sgd_20ep_resize_batch32
# destination_path = MAIN_PATH + 'Pytorch_ArcFace/final_data/fine_tuning_result/r34_Arc_sgd_20ep_resize_batch32' # r34_Combined_sgd_20ep_resize_batch64
# destination_path = MAIN_PATH + 'final_data/fine_tuning_result/r18_Cos_sgd_20ep_resize_batch32' # r18_Cos_sgd_20ep_resize_batch32
destination_path = MAIN_PATH + 'final_data/fine_tuning_result/r18_Arc_sgd_20ep_resize_batch32_adamw' # r18_Arc_sgd_20ep_resize_batch32_adamw


shutil.move(source_path, destination_path)

'/content/drive/MyDrive/딥러닝/RealTimeFaceRecognition/final_data/fine_tuning_result/r18_Arc_sgd_20ep_resize_batch32_adamw'